In [2]:
from jupyter_dash import JupyterDash
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go
import pandas as pd
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

In [3]:
# Load data
#df = pd.read_csv('all_reservoirs_data.csv', index_col=0, parse_dates=True)
df = pd.read_csv('all_wri_jan2011_dec2020.csv', index_col=0, parse_dates=True)
df = df.sort_values(by='DATE')
indiawris_df = pd.read_csv('all_indiawris_jan2001_may2021.csv',index_col=0, parse_dates=True)
indiawris_df = indiawris_df.sort_values(by='DATE')
indiawris_df.index = pd.to_datetime(indiawris_df['DATE'])
df.index = pd.to_datetime(df['DATE'],format='%Y-%m-%d')
df_sb = pd.read_csv('Dashboard_sidebar_options.csv')
resdata=pd.read_csv('india_states_reservoirs.csv')
resdata.head()

Unnamed: 0           State  Reservoirs
0           0     Maharashtra           1
1           1       Karnataka           1
2           2  Andhra Pradesh           1
3           3      Tamil Nadu           1
4           4          Kerala           1

In [4]:
indiawris_df

DATE  ACTUAL_MM  RES_LEVEL_M  PRESENT_STORAGE_BCM  \
DATE                                                                  
2001-01-01  2001-01-01       0.00      856.740                0.033   
2001-01-01  2001-01-01       2.60      694.740                0.194   
2001-01-01  2001-01-01       0.00      749.780                0.840   
2001-01-01  2001-01-01       0.00      885.810                0.553   
2001-01-02  2001-01-02       7.68      694.750                0.195   
...                ...        ...          ...                  ...   
2021-05-18  2021-05-18       4.39      860.085                0.068   
2021-05-19  2021-05-19      12.64      879.150                0.286   
2021-05-19  2021-05-19       3.13      860.112                0.068   
2021-05-19  2021-05-19       5.97      740.840                0.271   
2021-05-19  2021-05-19       0.61      689.970                0.140   

            TEMPERATURE_C  RES_LEVEL_FT  PRESENT_STORAGE_TMC  RESERVOIR  
DATE                                                                     
2001-01-01      23.400000   2810.826862             1.165384    Harangi  
2001-01-01      23.400000   2279.330782             6.851046     Kabini  
2001-01-01      23.400000   2459.908215            29.664323        KRS  
2001-01-01      23.400000   2906.200880            19.529013  Hemavathi  
2001-01-02      23.400000   2279.363590             6.886361     Kabini  
...                   ...           ...                  ...        ...  
2021-05-18      22.369371   2821.801271             2.401398    Harangi  
2021-05-19      21.078239   2884.350486            10.099996  Hemavathi  
2021-05-19      22.369371   2821.889854             2.401398    Harangi  
2021-05-19      21.411737   2430.577506             9.570276        KRS  
2021-05-19      21.973745   2263.681175             4.944054     Kabini  

[29304 rows x 8 columns]

In [5]:
geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"

In [6]:
cresdf = pd.read_csv('cauvery_reservoirs.csv')

In [7]:
cresdf['text'] = cresdf['Dam/Reservoir'] + '<br>Storage Capacity ' + (cresdf['Storage capacity (tmcft)']).astype(str)+' tmcft'
limits = [(0,10),(11,20),(21,30),(31,50),(51,70)]
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
reservoirs = []
scale = 4

In [8]:
#india map with reservoir locations
mapfig = go.Figure(data=go.Choropleth(
    geojson=geojson,
    #geojson= india_states,
    #featureidkey='properties.st_nm',
    featureidkey='properties.ST_NM',
    locationmode='geojson-id',
    locations=resdata['State'],
    z=resdata['Reservoirs'],
    autocolorscale=False,
    #colorscale='inferno',
    marker_line_color='white' ,
    #locations='Area_Name',
                  #animation_frame='State',
      #color='Active_Cases'
    
    colorbar=dict(
             #title={'text':"Reservoirs Count"},
              thickness=1,
              len=0.3,
              bgcolor= 'rgba(255,255,255,0.0)',
              tick0=0,
               dtick=20000,
               xanchor='left',
              x=0.01,
               yanchor='bottom',
               y=0.05
               )
               
    
))


mapfig.update_layout(
    title=dict(
            #text="Reservoirs in India",
            xanchor='center',
            x=0.5,
            yref='paper',
            yanchor='bottom',
            y=1,
            pad={'b':10}),
            margin={'r':0,'t':30,'l':0,'b':0},
            height=550,
            width=550
      )





###adding reservoir bubbles to map


for i in range(len(cresdf)):
    lim = limits[i]
    df_sub = cresdf.iloc[i]
    mapfig.add_trace(go.Scattergeo(
        lon = [df_sub['Long']],
        lat = [df_sub['Lat']],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['Storage capacity (tmcft)'] / scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))
    
mapfig.update_geos(
         visible=False,
         projection=dict(
                   type='conic conformal',
                    parallels=[12.4729,35.1728055],
                   rotation={'lat':24,'lon':80}),
         lonaxis={'range':[68,98]},
         lataxis={'range':[6,38]})

mapfig.update_layout(
        title_text = '<br>Cauvery Reservoirs<br>(Click legend to toggle traces)',
        font = {"size": 9, "color":"White"},
        titlefont = {"size": 15, "color":"White"},
        showlegend = True,
        geo=dict(bgcolor= 'rgba(0,0,0,0)', lakecolor='#4E5D6C',
                                          landcolor='rgba(51,17,0,0.2)',
                                          subunitcolor='grey',
                                          scope = 'asia',),
       # geo = dict(
        #    scope = 'asia',
         #   landcolor = 'rgb(217, 217, 217)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        )
    #)


In [10]:




# Initialize the app
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True

#get all reservoir names as dictionary options
def get_options(list_reservoirs):
    dict_list = []
    for i in list_reservoirs:
        dict_list.append({'label': i, 'value': i})

    return dict_list

#get other sidebar options from csv file
def get_sidebar_options(data,opt):
    optdict = data.loc[data['Option']== opt,['Key','Value']].to_dict()
    return [{'label':optdict['Key'][i], 'value':str(i)} for i in optdict['Key']]



tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold',
    'color': 'grey',
    'backgroundColor': '#1E1E15'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#1E1E1E',
    'color': 'lightblue',
    'padding': '6px'
}


#chart elements definition

#reservoirs dropdown
reservoirs_dropdown = dbc.Col([
                                    # className='div-for-dropdown',
                                     #children=[
                                        dcc.Dropdown(id='resselector_dpdn', options=get_options(df['RESERVOIR'].unique()),
                                                      multi=True, value=[df['RESERVOIR'].sort_values()[0]],
                                                      style={'backgroundColor': '#1E1E1E'},placeholder="Select a Reservoir",
                                                      className='resselector_dpdn'
                                                      )
                                        ,
                                     #],
                                    ],
                                     style={'color': '#1E1E1E','margin-right': '2em','width':'50%','display':'inline-block'}
                                    )

#algorithm dropdown
algorithm_dropdown = dbc.Col([
                                    # className='div-for-dropdown',
                                    # children=[
                                         dcc.Dropdown(id='algorithm_dpdn', options=get_sidebar_options(df_sb,'Algorithm'),
                                                      multi=True, placeholder="Select a Model",
                                                      style={'backgroundColor': '#1E1E1E'},
                                                      className='algorithm_dpdn'
                                                      ),
                                     #],
                                    ],
                                     style={'color': '#1E1E1E','margin-right': '2em','width':'50%','display':'inline-block'})
#forecast_button
forecast_btn =                     dbc.Col([
                                            html.Button('Forecast',id='forecast_btn',
                                                     style={'width': '150px','font':'16px','fontColor':'white'}),
                                            ])

#actualforecast radiobtn
actualforecast_radiobtn = dbc.Col([
                                     
                                         dcc.RadioItems(id='actualforecast_rb', 
                                                     options = [
                                                                {'label':'Actual', 'value':'True'},
                                                                {'label':'Forecast', 'value':'False'}
                                                            ],
                                                            value = 'True',
                                                      style={'backgroundColor': '#1E1E1E'},
                                                      className='actualforecast'
                                                      ),
                                     ],
                                     style={'margin-left': '20em','width':'100px','display':'inline-block'})

#wri_indiawris_label
wri_indiawris_label =                  dbc.Col([
                                            html.Label("Dataset: "),],
                                            style={'margin-right': '0em','width':'50px','display':'inline-block'},align="center")


#wri_indiawris radiobtn
wri_indiawris_radiobtn = dbc.Col([
                                     
                                         dcc.RadioItems(id='wri_indiawris_rb', 
                                                     options = [
                                                                {'label':'WRI', 'value':'True'},
                                                                {'label':'IndiaWRIS', 'value':'False'}
                                                            ],
                                                            value = 'True',
                                                      style={'backgroundColor': '#1E1E1E'},
                                                      className='wri_indiawris'
                                                      ),
                                     ],
                                     style={'margin-right': '2em','width':'100px','display':'inline-block'})



#timeperiod_label
timeperiod_label =                  dbc.Col([
                                            html.Label("No.of Days (1-90): "),],
                                            style={'margin-right': '0em','width':'120px','display':'inline-block'},align="center")
#timeperiod dropdown
timeperiod_number =  dbc.Col([
                                     #className='div-for-dropdown',
                                     #children=[
                                         dcc.Input(
                                                    id="timeperiod_number", type="number",
                                                     min=1, max=90, step=1,style={'backgroundColor': '#1E1E1E','color': 'yellow'}),
                                         
                                     ],
                                     style={'width': '150px','margin-right': '1em','display':'inline-block'})
#featuretoggle dropdown
uni_multi_radiobtn = dbc.Col([
                                     #className='div-for-dropdown',
                                     #children=[
                                         dcc.RadioItems(id='uni_multi_rb', # options=get_sidebar_options(df_sb,'Feature Toggle'),
                                                      #multi=False, placeholder="Select Feature Toggle",
                                                     options = [
                                                                {'label':'Univariate(without Weather data)', 'value':'True'},
                                                                {'label':'Multivariate(with Weather data)', 'value':'False'}
                                                            ],
                                                            value = 'True',
                                                      style={'backgroundColor': '#1E1E1E'},
                                                      className='uni_multi_rb'
                                                      ),
                                     ],
                                     style={'margin-right': '2em','width':'250px','display':'inline-block'})
#plot chart
plot_chart=                    html.Div(className='six columns div-for-charts bg-grey',
                             children=[
                                  dcc.Tabs(
                                      children=[
                                    dcc.Tab(label='Line Plots', style=tab_style, selected_style=tab_selected_style,
                                        children=[
                                            dcc.Graph(id='timeseries', config={'displayModeBar': False}, animate=True)
                                                ]),
                                      
                                    dcc.Tab(label='Box Plots', value='tab-2', style=tab_style, selected_style=tab_selected_style,
                                        children=[
                                            dcc.Graph(id='timeseries1', config={'displayModeBar': False}, animate=True)
                                                 ]), 
                                                  ], style=tabs_styles,
                                                                              
                                  
                                  ),                                
                                 
                                 
                             ])
#india map 
india_map =  html.Div(className='six columns div-for-charts bg-grey',
                             children=[
                                 dcc.Graph(figure=mapfig)
                             ])

# design html interface
dropdowns = html.Div(
    children=[
        html.Div(className='column',
                 children=[html.Center(html.H2('CAUVERY RESERVOIRS WATER LEVEL FORECASTING SYSTEM')),
                    html.Div(
                             children=[
                                dbc.Row([ 
                                 reservoirs_dropdown ,

                                 ## adding sidebar options
                                 algorithm_dropdown ,
                                 
                                 ## end side bar options
                                ], style={'display':'flex'}
                             ) ,
                             ],),
                           html.Br(),
                     html.Div(
                             children=[
                                 dbc.Row([ 
                                 actualforecast_radiobtn,
                                 wri_indiawris_label,    
                                 wri_indiawris_radiobtn,  
                                 uni_multi_radiobtn,
                                 timeperiod_label,
                                 timeperiod_number ,
                                 forecast_btn
                                 
                                 ## end side bar options
                                ], style={'display':'flex'}
                             ) ,
                             ],),
                     html.Div(id="graph-container", 
                             children=[
                                 
                                 india_map,
                                 plot_chart
                                 
                                 
                             ]),
                ]),
            ])
 

app.layout = html.Div(
    [dropdowns])


# Reset reservoir and algorithm dropdown options on change of wri/indiawris dataset or actual/forecast radio buttons
@app.callback(Output('resselector_dpdn', 'value'),
              Output('algorithm_dpdn', 'value'),
              [Input('wri_indiawris_rb','value'),
               Input('actualforecast_rb', 'value')])
def reset_dropdowns(value1, value2):
    return "",""
              
# Callback for reservoir timeseries 
@app.callback(Output('timeseries', 'figure'),
              Output('timeseries1', 'figure'),
              [Input('resselector_dpdn', 'value'),
              Input('wri_indiawris_rb','value')])
def update_graph(reservoir_list, wri_indiawris):
    #if len(reservoir_list) ==0: 
     #  return {}
    trace1 = [] #line plot
    trace2 = [] #box plot
    df_sub = None
    plot_title = 'Reservoir Water Levels in TMC'
    if wri_indiawris == 'True':
        df_sub = df
        plot_title = plot_title+  '(WRI) '
    else:
        df_sub = indiawris_df
        plot_title = plot_title+ '(IndiaWRIS) '
    #df_sub = df
    #plot_title = 'Reservoir Water Levels in TMC'
    for reservoir in reservoir_list:
        trace1.append(go.Scatter(x=df_sub[df_sub['RESERVOIR'] == reservoir].index,
                                 y=df_sub[df_sub['RESERVOIR'] == reservoir]['PRESENT_STORAGE_TMC'],
                                 mode='lines',
                                 opacity=0.7,
                                 name=reservoir,
                                 textposition='bottom center'))
       # fig = go.Figure()
        trace2.append(go.Box(y=df_sub[df_sub['RESERVOIR'] == reservoir]['PRESENT_STORAGE_TMC'], name=reservoir))
        


    linetraces = [trace1]
    data1 = [val for sublist in linetraces for val in sublist]
    figure1 = {'data': data1,
              'layout': go.Layout(
                  colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 15},
                  hovermode='x',
                  autosize=True,
                  title={'text': plot_title, 'font': {'color': 'yellow'}, 'x': 0.5},
                  xaxis={'range': [df_sub.index.min(), df_sub.index.max()],
                        'rangeslider': {'visible':True},
                        'rangeselector': {'visible':True, 'bgcolor':"blue", 'buttons':list([
                            dict(count=1, label="1y",   step="year", stepmode="backward"),
                            dict(count=3, label="3y",  step="year", stepmode="backward"),
                            dict(count=5, label="5y",  step="year", stepmode="backward"),
                            dict(step="all") 
                                   ])}                        
                         },
                    ),

              }
    
    boxtraces = [trace2]
    data2 = [val for sublist in boxtraces for val in sublist]
    figure2 = {'data': data2,
              'layout': go.Layout(
                  colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 15},
                  hovermode='x',
                  autosize=True,
                  title={'text': plot_title, 'font': {'color': 'yellow'}, 'x': 0.5},
                    ),

              }

    return figure1,figure2


#toggle graph container and multi option of reservoir dropdown based on actual/forecast radio button
@app.callback(Output('graph-container', 'style'),
              Output('resselector_dpdn', 'multi'), 
              [Input('actualforecast_rb', 'value')])
def hide_graph(inputval):
    if inputval == 'True':
        return {'display':'block'},True
    return {'display':'none'},False
#toggle button to disable/enable other html controls based on actual/forecast radio button
@app.callback(Output('forecast_btn', 'disabled'),
              Output('timeperiod_number', 'disabled'),
              Output('algorithm_dpdn', 'disabled'),
             [Input('actualforecast_rb', 'value')])
def set_button_enabled_state(inputval):
        if inputval == 'True':
            return True, True, True
        return False, False, False

app.run_server(mode='external',debug=True, port=8088)

Dash app running on http://127.0.0.1:8088/


#high level forecast function to be called from dashboard which inturn calls univatiate and multivariate model functions
def forecast(uni_multi,wri_indiawris, reservoir_list, model_list, noofdays ):
    '''
        uni_multi -     boolean, if true then univariate else multivariate models to be used
        wri_indiawris - boolean, if true wri datasets else indiawris datasets to be used
        reservoir_list - list of selected reserviors for forecasting
        model_list     - list of models(algorithms) to be used for forecasting
        noofdays       - number of days to be forecasted
        
    '''
    data =''
    if wri_indiawris: #this is boolean variable, wri is true use wri datasets, else use indiawris datasets
        data = 'wri'
    else:
        data = 'indiawris'
        
    if uni_multi: # boolean variable, univariate is true, so use univariate models, else multivariate models
        #univariate_models(data, reservoir_list, model_list, noofdays)
        # load each model and reservoir and store the predition
        for model in model_list:
            for reservoir in reservoir_list:
                result = get_model_prediction_result(data, model, reservoir, noofdays)
                store_result_in_df(result)
    else:
        #multivariate_models(data, reservoir_list, model_list, noofdays)
         # load each model and reservoir and store the predition
        for model in model_list:
            for reservoir in reservoir_list:
                result = get_model_prediction_result(data, model, reservoir, noofdays)
                store_result_in_df(result)
                
    final_result = store_result_in_df #dataset with all model results
    return final_result